#### Pipeline ETL para ingesta en base de datos relacional

In [1]:
# importamos las librerias necesarias
import pandas as pd
import json
import os
import datetime
import matplotlib.pyplot as plt
import numpy as np
import gzip

In [2]:
# Cambiar el directorio de trabajo al directorio donde se encuentran los archivos JSON
%cd ./metadata-sitios
# Obtener una lista de archivos en el directorio actual
archivos = os.listdir()

# Crear un DataFrame vacío para almacenar los datos
df_metadata = pd.DataFrame()

# Iterar sobre cada archivo en la lista de archivos
for archivo in archivos:
    # Verificar si el archivo tiene la extensión .json
    if archivo.endswith('.json'):
        # Leer el archivo JSON y cargarlo en un DataFrame
        data = pd.read_json(archivo, lines=True)
        # Concatenar los datos del archivo al DataFrame principal
        df_metadata = pd.concat([df_metadata, data], axis=0)
print("finalizo cargan de datos")

c:\Users\sebas\OneDrive\Escritorio\Proyecto-Grupal-Google-yelp\Sprint 2 Ingenieria de Datos\pipeline ETL\google\metadata-sitios
finalizo cargan de datos


In [3]:
df_metadata.head(2)

,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url
0,Porter Pharmacy,"Porter Pharmacy, 129 N Second St, Cochran, GA ...",0x88f16e41928ff687:0x883dad4fd048e8f8,None,32.388300,-83.35710,[Pharmacy],4.9,16,None,"[[Friday, 8AM–6PM], [Saturday, 8AM–12PM], [Sun...","{'Service options': ['In-store shopping', 'Sam...",Open ⋅ Closes 6PM,"[0x88f16e41929435cf:0x5b2532a2885e9ef6, 0x88f1...",https://www.google.com/maps/place//data=!4m2!3...
1,City Textile,"City Textile, 3001 E Pico Blvd, Los Angeles, C...",0x80c2c98c0e3c16fd:0x29ec8a728764fdf9,None,34.018891,-118.21529,[Textile exporter],4.5,6,None,None,None,Open now,"[0x80c2c624136ea88b:0xb0315367ed448771, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...


In [4]:
# Eliminamos columnas irrelevantes
df_metadata.drop(columns=['address', 'state', 'relative_results', 'url','hours','description', 'price'], inplace=True)

In [5]:
from pandas import json_normalize 

# Desanidar los datos de la columna "MISC"
df_desanidado = json_normalize(df_metadata['MISC'])

In [6]:
# Vamos a trabajar con una copia de metadata para crear una posible tabla "miselaneos"
gmap_id = df_metadata.copy()

# Resetear el índice del DataFrame df_gmap_id
gmap_id.reset_index(drop=True, inplace=True)

# Combinar los DataFrames desanidados
miselaneos = pd.concat([gmap_id, df_desanidado], axis=1)

In [7]:
# Eliminamos columna MISC
df_metadata.drop(columns=['MISC'], inplace=True)

In [8]:
# DEBERIAMOS QUEDARNOS UNICAMENTE CON EL PRIMER VALOR DE LA COLUMNA CATEGORY
df_metadata['category'] = df_metadata['category'].str[0]


In [9]:
df_metadata['category']

0                        Pharmacy
1                Textile exporter
2               Korean restaurant
3                    Fabric store
4                    Fabric store
                   ...           
274996                 Restaurant
274997           Trucking company
274998               Beauty salon
274999    Truck accessories store
275000                 Campground
Name: category, Length: 2200008, dtype: object

In [10]:
# Verificar si hay valores duplicados
df_metadata.duplicated().value_counts()

False    2173428
True       26580
Name: count, dtype: int64

In [11]:
# Eliminamos duplicados
df_metadata.drop_duplicates(inplace=True)

In [12]:
# Verificar si hay valores nulos
df_metadata.isnull().sum()

name                 31
gmap_id               0
latitude              0
longitude             0
category          13945
avg_rating            0
num_of_reviews        0
dtype: int64

In [13]:
df_metadata.shape

(2173428, 7)

In [14]:
# Cambio el nombre de la columna gmap_id a business_id 
df_metadata.rename(columns={'gmap_id': 'business_id'}, inplace=True)

In [15]:
# Verificamos tamaño de dataset
df_metadata.shape

(2173428, 7)

In [16]:
# Observamos resultados
df_metadata.head(5)

,name,business_id,latitude,longitude,category,avg_rating,num_of_reviews
0,Porter Pharmacy,0x88f16e41928ff687:0x883dad4fd048e8f8,32.388300,-83.357100,Pharmacy,4.9,16
1,City Textile,0x80c2c98c0e3c16fd:0x29ec8a728764fdf9,34.018891,-118.215290,Textile exporter,4.5,6
2,San Soo Dang,0x80c2c778e3b73d33:0xbdc58662a4a97d49,34.058092,-118.292130,Korean restaurant,4.4,18
3,Nova Fabrics,0x80c2c89923b27a41:0x32041559418d447,34.023669,-118.232930,Fabric store,3.3,6
4,Nobel Textile Co,0x80c2c632f933b073:0xc31785961fe826a6,34.036694,-118.249421,Fabric store,4.3,7


In [22]:
# Importamos df_florida para quedarnos solo con los datos que coinciden con esa tabla
path = r"C:\Users\sebas\OneDrive\Escritorio\Proyecto-Grupal-Google-yelp\Sprint 2 Ingenieria de Datos\pipeline ETL\google\florida_data\reviews.csv"
df_florida = pd.read_csv(path)

In [24]:
# Nos quedamos con los datos de metada que solo coincidan con df_florida en business_id
df_merged = pd.merge(df_metadata, df_florida, on='business_id', how='inner')

In [27]:
# Vemos que tenemos considerablemente menos datos que antes (2173428 vs 77578)
df_merged.shape

(77578, 11)

In [28]:
df_merged.columns

Index(['name', 'business_id', 'latitude', 'longitude', 'category',
       'avg_rating', 'num_of_reviews', 'user_id', 'rating', 'timestamp',
       'sentiment_analysis'],
      dtype='object')

In [29]:
# Ahora eliminamos las columnas resultantes del merge 
df_merged.drop(columns=['user_id', 'rating', 'timestamp', 'sentiment_analysis'], inplace=True)

In [31]:
# vemos el resultado final
df_merged.head(5)

,name,business_id,latitude,longitude,category,avg_rating,num_of_reviews
0,Baker Distributing Company,0x88c33fb303dc8e69:0x3485e731f1a704f8,27.401269,-82.532346,Air conditioning system supplier,4.3,8
1,Cutting Point,0x88909517e0c1c69b:0xeaa131bf8f6f1251,30.531940,-87.242073,Hair salon,4.7,65
2,Cutting Point,0x88909517e0c1c69b:0xeaa131bf8f6f1251,30.531940,-87.242073,Hair salon,4.7,65
3,Cutting Point,0x88909517e0c1c69b:0xeaa131bf8f6f1251,30.531940,-87.242073,Hair salon,4.7,65
4,Dexter Coin Laundry,0x88dd12df337bdd49:0xcdba96fd79ced708,28.032902,-81.733152,Laundromat,3.2,13


In [32]:
# Exportamos el dataset
df_merged.to_csv('business.csv', index=False)